In [ ]:
import pandas as pd
import numpy as np
import random
from util import set_item, usage_masking, error_cal, error_masking, Processing, damage_sum, damage_percent, volume_checker, sampler, sampler_rate, model_checker, money_sum
from model import Kcluster, Let_model
import progressbar

In [ ]:
df = pd.read_csv('./2023공모전데이터.csv')

# 몇 번의 시행을 할 것인지 설정함
num_search = 5000

for num in range(num_search):
    Power_list = list(df.Power_Code.unique())
    model = Kcluster(Power_list, df)

    # 모델에서 사용하는 Hyperparameter가 다음 값으로 랜덤하게 설정되도록 함.
    # 랜덤하게 설정된 Hyperparameter를 통해 모델을 수행. 이러한 반복을 num_search만큼 진행함
    random_k = random.randint(3, 9)
    random_first_search = random.randint(50, 300)
    random_min_length = random.randint(3, 12)
    random_max_volume = (197799 / random_k) * (random.uniform(1.0, 1.13))

    result = Let_model(Power_list, model, random_k, random_first_search, random_min_length, random_max_volume, df, 'high')
    check_value = 0
    for i in result:
        total, medium, small = model_checker(i, df)
        if total >= 20000:
            check_value = check_value + 1

        # 소형이 중형보다 크다는 조건을 만족하는 것이 불가능하여 삭제
        # if small >= medium:
        #     check_value = check_value + 1

    if check_value == random_k:
        print('Collect Result')

        temp_df = pd.DataFrame()
        for i, value in enumerate(result):
            real_money, best_money, percent = money_sum(value, df)
            temp_df.loc[i, 'item_complex'] = str(value)
            temp_df.loc[i, 'Total_volume'] = str(model_checker(value, df))
            temp_df.loc[i, 'Total_Money'] = real_money
            temp_df.loc[i, 'Total_Best_Money'] = best_money
            temp_df.loc[i, 'Money_percent'] = percent

        temp_df.to_csv(f'./data2/result_{num}.csv', index=False)
    else:
        print('Not to match. check_value is {}, and value is {}'. format(check_value, (random_k)))